### Ingest drivers.json file

##### Read the nested json file using spark df reader

In [0]:
dbutils.widgets.text("p_data_source","")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType
from pyspark.sql.functions import col, current_timestamp, lit, to_timestamp,concat

In [0]:
name_schema = StructType(fields=[StructField("forename", StringType(), True), 
                                 StructField("surname", StringType(), True)])

In [0]:
drivers_schema = StructType(fields=[StructField("driverId", IntegerType(), False), 
                                  StructField("driverRef", StringType(), True),
                                  StructField("number", IntegerType(), True),
                                  StructField("code", StringType(), True),
                                  StructField("name", name_schema),
                                  StructField("dob", DateType(), True),
                                  StructField("nationality", StringType(), True),
                                  StructField("url", StringType(), True)])

In [0]:
drivers_df = spark.read \
.option("header", "true") \
.schema(drivers_schema) \
.json(f"{raw_folder_path}/drivers.json")

In [0]:
drivers_df.printSchema()

##### Select the columns

In [0]:
drivers_with_columns_df = drivers_df.withColumnRenamed("driverId", "driver_id") \
                                    .withColumnRenamed("driverRef", "driver_ref") \
                                    .withColumn("ingestion_date", current_timestamp()) \
                                    .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname"))) \
                                    .withColumn("datasource", lit(v_data_source))

In [0]:
drivers_final_df = drivers_with_columns_df.drop("url")
                                          

Write the result as parquet file

In [0]:
#drivers_final_df.write.mode("overwrite").parquet(f"{processed_folder_path}/drivers")

In [0]:
drivers_final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.drivers")

In [0]:
#%fs
#ls /mnt/formula1dlstudy/processed/drivers

In [0]:
%sql
SELECT * FROM f1_processed.drivers

In [0]:
display(spark.read.parquet(f"{processed_folder_path}/drivers"))

In [0]:
dbutils.notebook.exit("Success")